# Система скоринга на данных ИНТЕР РАО

Ноутбук для хакатона Energy Data Science Challange B2B

Идея монетизации данных: построить скоринговую модель на данных Интер РАО

## Подготовка данных

Для примера берем 150 человек клиентов и выгржаем для них таблицы клиентов. По каждой таблице кодируем категориальные переменные. Затем групируем данные за период в одну строку используя разные виды группировок. Таким образом на каждого клиента мы получаем одну строку с набором переменных.

In [40]:
import pandas as pd
from sklearn import preprocessing

In [41]:
def generate_features(df, id_col):

    df_mean = df.groupby(id_col).mean()
    df_max =df.groupby(id_col).max()
    df_min =df.groupby(id_col).min()
    df_std = df.groupby(id_col).std()
    
    df = df_mean.join(df_max, how='left', lsuffix = '_mean', rsuffix = '_max')
    df = df.join(df_min, how='left', rsuffix = '_min')
    df = df.join(df_std, how='left', rsuffix = '_std')
    return df

In [42]:
def encode_cat_features(df, cat_columns):
    for col in cat_columns:
        le = preprocessing.LabelEncoder()
        df[col]=df[col].astype(str)
        df[col] = le.fit_transform(df[col])
    return df

In [43]:
#таблица 1
real = pd.read_csv('ch.xackaton_bitovie_abon_nach_dz_kz_oplata_realisatsiya.csv')
real=real.drop('month_year',axis=1)
real =generate_features(real, 'id')
print(real.shape)
real.head(3)

(150, 40)


,dz_for_period_begin_rub_mean,kz_for_period_begin_rub_mean,charged_total_kVth_mean,charged_total_rub_mean,user_pay_rub_mean,realization_rub_mean,current_year_realization_rub_mean,current_period_realization_rub_mean,dz_for_period_end_rub_mean,kz_for_period_end_rub_mean,...,dz_for_period_begin_rub_std,kz_for_period_begin_rub_std,charged_total_kVth_std,charged_total_rub_std,user_pay_rub_std,realization_rub_std,current_year_realization_rub_std,current_period_realization_rub_std,dz_for_period_end_rub_std,kz_for_period_end_rub_std
id,,,,,,,,,,,,,,,,,,,,,
3003,1906.5616,-9.4984,368.56,2089.7840,2346.9976,2036.3864,364.9820,130.4384,1959.9592,-9.4984,...,557.335548,47.492000,57.301163,307.475406,1624.479542,283.822228,754.066832,455.676091,495.577268,47.492000
3006,1274.1104,-12.3352,120.24,682.2148,1252.0024,640.5132,521.9364,114.1136,1315.8120,-12.3352,...,1096.818720,54.122195,115.309106,647.184504,2770.840567,459.258850,352.526559,213.356884,1078.218237,54.122195
3009,1157.1816,-114.0604,143.76,812.3980,1515.0868,821.8032,391.5796,120.7680,1147.7764,-129.3348,...,1580.101259,260.295039,230.082463,1285.475719,2630.168852,824.463637,1181.371231,1030.556695,1586.505047,264.493095


In [44]:
# таблица 2
dogovor = pd.read_csv('ch.xackaton_fuv2_bitovie_abon_parametri_dogovora.csv')
dogovor = dogovor .set_index('id')
dogovor =encode_cat_features(dogovor, dogovor.columns)
print(dogovor.shape)
dogovor.head(3)

(150, 14)


,ownerhip_form,coliving_flag,home_running_way,type_ls_building,counter_type,counter_category,pu_setup_place,pu_actual_state,odn_is_valuable,calling_is_on,autocalling_is_on,account_type,epd_sign,epd_sign_1
id,,,,,,,,,,,,,,
3003,2,0,1,0,17,2,4,2,0,0,0,1,2,2
3006,5,0,1,0,0,2,4,2,0,0,0,1,2,2
3009,5,0,1,0,28,2,4,2,0,0,0,1,2,2


In [45]:
#таблица 3
nachisl =pd.read_csv('ch.xackaton_po_bitovie_abon_nachisleniya_2019_2021.csv')
nachisl=encode_cat_features(nachisl, ['day_period','stove_type','tariff','voltage'])
nachisl =nachisl.drop('month_year',axis=1)
nachisl =generate_features(nachisl, 'id')
print(nachisl.shape)
nachisl.head(3)

(150, 24)


,day_period_mean,stove_type_mean,tariff_mean,voltage_mean,used_energy_capacity_kVth_mean,used_energy_cost_with_nds_rub_mean,day_period_max,stove_type_max,tariff_max,voltage_max,...,tariff,voltage,used_energy_capacity_kVth,used_energy_cost_with_nds_rub,day_period_std,stove_type_std,tariff_std,voltage_std,used_energy_capacity_kVth_std,used_energy_cost_with_nds_rub_std
id,,,,,,,,,,,,,,,,,,,,,
3003,0.0,0.0,1.0,0.0,368.56,2089.7840,0,0,1,0,...,1,0,250,1333.30,0.0,0.0,0.0,0.0,57.301163,307.475406
3006,0.0,0.0,1.0,0.0,120.24,682.2148,0,0,1,0,...,1,0,-385,-2158.78,0.0,0.0,0.0,0.0,115.309106,647.184504
3009,0.0,0.0,1.0,0.0,143.76,812.3980,0,0,1,0,...,1,0,-698,-3884.11,0.0,0.0,0.0,0.0,230.082463,1285.475719


Здесь мы использовали самые простые фичегенрации. В реальности можно создать гораздо больше полезных признаков, например групповые признаки, максимальная длительность просрочки оплаты и.т.д.

## Совместная модель скоринга

In [46]:
df = pd.concat([real,dogovor,nachisl],axis=1)
indexs = df.index
print(df.shape)
df.head()

(150, 78)


,dz_for_period_begin_rub_mean,kz_for_period_begin_rub_mean,charged_total_kVth_mean,charged_total_rub_mean,user_pay_rub_mean,realization_rub_mean,current_year_realization_rub_mean,current_period_realization_rub_mean,dz_for_period_end_rub_mean,kz_for_period_end_rub_mean,...,tariff,voltage,used_energy_capacity_kVth,used_energy_cost_with_nds_rub,day_period_std,stove_type_std,tariff_std,voltage_std,used_energy_capacity_kVth_std,used_energy_cost_with_nds_rub_std
id,,,,,,,,,,,,,,,,,,,,,
3003,1906.5616,-9.4984,368.56,2089.7840,2346.9976,2036.3864,364.9820,130.4384,1959.9592,-9.4984,...,1,0,250,1333.30,0.0,0.0,0.0,0.0,57.301163,307.475406
3006,1274.1104,-12.3352,120.24,682.2148,1252.0024,640.5132,521.9364,114.1136,1315.8120,-12.3352,...,1,0,-385,-2158.78,0.0,0.0,0.0,0.0,115.309106,647.184504
3009,1157.1816,-114.0604,143.76,812.3980,1515.0868,821.8032,391.5796,120.7680,1147.7764,-129.3348,...,1,0,-698,-3884.11,0.0,0.0,0.0,0.0,230.082463,1285.475719
3012,476.7768,-0.0944,70.80,402.2612,545.7332,402.2612,103.2376,60.3144,476.7768,-0.0856,...,1,0,-264,-1467.84,0.0,0.0,0.0,0.0,91.676969,510.046518
3015,272.3872,-38.6736,76.80,436.5840,517.0800,418.5568,215.8464,163.6104,290.4144,-35.1952,...,1,0,17,94.52,0.0,0.0,0.0,0.0,28.730066,159.733051


Таким образом, у нас есть минимум 78 признаков на которых можно строить скоринговую модель. Не хватает разметки о клиентах, которую можно получить в совместном проекте с банком или другой организацией занимающейся кредитным скорингом.

## Предоставление признаков для построение скоринговых моделей внешних клиентов

По понтным причинам компания не может предоставлять персональные данные из таблицы сторонним орагнизациям. Один из способов сохранит ценность данных для построения моделей, но скрыть их сущностную составляющую это применить к данным снижение размерности, например метод основных компонент (pca).

Этот математический метод представляет данны таблицы в виде меньшего числа колонок, с потерей доли информации. Интерпертировать значения нового датасета  - невозможно, восстановить датасет после применения снижения пространства - невозможно. 

При этом партнер сможет строить свои модели на этих данных. 

In [47]:
from sklearn.decomposition import PCA

In [48]:
new_datasets = []
for dataset in [real,dogovor,nachisl]:
    pca  = PCA(0.99) #мы сохраняем 99%информации
    print('Размер датасета до применения РСА: ', dataset.shape)
    new_set = pca.fit_transform(dataset) 
    """
  В проде, конечно, нужно тренировать рса на отдельном отложенном ~миллионе строк
  и категориальные признаки стоит представлять отдельно, т.е разбивать датасет
  на большее количество частей, нормализовывать и.т.д.
  Здесь представлен очень упрощенный пример.
    """              
    print('Размер датасета после применения РСА: ', new_set.shape)
    new_datasets.append(new_set)

Размер датасета до применения РСА:  (150, 40)
Размер датасета после применения РСА:  (150, 8)
Размер датасета до применения РСА:  (150, 14)
Размер датасета после применения РСА:  (150, 4)
Размер датасета до применения РСА:  (150, 24)
Размер датасета после применения РСА:  (150, 2)


In [50]:
df = pd.concat([pd.DataFrame(i) for i in new_datasets ] ,axis=1)
df.columns = ['feature_'+str(i) for i in range(df.shape[1])]
print(df.shape)
df.index = indexs
df.head()

(150, 14)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13
id,,,,,,,,,,,,,,
3003,-5900.638547,-3330.871205,1850.188203,337.198358,-1327.626704,-2611.181228,-254.050852,-2520.579379,5.266193,0.428767,-0.418334,0.243046,-3689.935411,1489.542712
3006,-3367.184428,-3219.208357,2073.446941,-34.348042,-5737.914048,-4400.660208,2230.326521,-1454.814576,-11.670135,-2.732534,-0.464355,-0.680955,-2565.454413,-2528.427282
3009,-647.141686,-2081.280018,2453.842064,-686.796826,-7538.387062,606.272587,319.002184,-149.212647,16.292546,-2.426559,0.030750,0.422944,-716.129586,-3243.336719
3012,-6122.682277,-5157.104738,-4524.457713,-2125.687471,-119.441598,1241.214839,-807.950313,691.202147,-6.690470,-1.686052,-0.460803,-0.399238,-2837.298281,-1869.425491
3015,-9536.410392,-4753.280490,-4122.728607,-2488.924336,990.370698,-212.854884,13.341828,-122.928250,-10.739824,2.237611,-0.870970,-0.218569,-4315.763825,-1046.752512


Таким образом из 78 мы получили 14 неинтерпертируемых признаков которые можно продавать сторонним клиентам что бы они строили на этих данных свои модели.

## Пример реализации API

In [54]:
def fake_api():
    print('API запрашивает адрес:')
    inp = input()
    #происходит метчинг ид по адресу...
    inp = 3003 #:) всегда возвращает по первому клиенту
    print('Вовзращаю данные...')
    print(df[df.index==inp])
    print('Списано 4 рубля, остаток 2000200руб.')
    

In [55]:
fake_api()

API запрашивает адрес:
Москва, Прсопект Мира дом 4, квартира 21
Вовзращаю данные...
        feature_0    feature_1    feature_2   feature_3    feature_4  \
id                                                                     
3003 -5900.638547 -3330.871205  1850.188203  337.198358 -1327.626704   

        feature_5   feature_6    feature_7  feature_8  feature_9  feature_10  \
id                                                                             
3003 -2611.181228 -254.050852 -2520.579379   5.266193   0.428767   -0.418334   

      feature_11   feature_12   feature_13  
id                                          
3003    0.243046 -3689.935411  1489.542712  
Списано 4 рубля, остаток 2000200руб.
